# Crimson Responses - kilosort

basically the same thing as the other crimson response stuff, except we're working with kilosort stuff

In [ ]:
from matplotlib import pyplot as plt
from matplotlib.patches import Polygon
import numpy as np
import pandas as pd
from scipy import signal
from sklearn.decomposition import PCA
import os, glob, re
import openephys_utils 
from tqdm.notebook import tqdm
import kilosort

%matplotlib qt


batch run through kilosort on filtered data

october 2nd

In [ ]:
base_dir = 
directory = [directory for directory in os.listdir()]